# Web scraping pipeline Part 3

In this section, we finally are able to apply Natural Language Processing techniques to the dataset. Firstly, we connect the scraped Data with their corresponding rows of Courses Dataset. Using regular expressions, tokenization, lemmatization we extract key words from the name, description and requirements of courses.

In [1]:
import re
import json
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Natural Language Processing packages:

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer 
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

Language detection:

In [3]:
# !{sys.executable} -m pip install langdetect
from langdetect import detect

Reading scraped Data and course Dataset:

In [4]:
df_scrapedData = pd.read_csv('../Data/interim/df_scrapedAllData.csv')

In [5]:
df_scrapedData.head()

,description,rating,audience,counter,course,requirements,language
0,Accounting is one of the most important skills...,4.6,['Aspiring Accountants and Financial Analysts'...,3663.0,640100.0,No prior knowledge of accounting is assumed or...,English
1,This course is an introduction to the financia...,3.6,"['Students in business and Finance', 'Auditors...",33.0,385604.0,some knowledge of accounting,English
2,*Course Fully Updated for May 2019*The don’t c...,4.7,['Anyone interested in earning an extra income...,300.0,834836.0,You will need some basic knowledge of stock an...,English
3,This Mortgage Acceleration course will teach y...,3.7,['This Mortgage Acceleration course is designe...,7.0,504620.0,Students will need a reliable computer and int...,English
4,"This course is for bookkeepers, accountants an...",3.9,['Individuals / Directors who want to submit t...,10.0,359926.0,It would be helpful if you understood accounti...,English


In [6]:
df_coursesSampling = pd.read_csv('../Data/interim/df_samples.csv')

In [7]:
df_coursesSampling.head()

,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,contentInfo,publishedTime,category,timeSpent,publishDate,level,paidBool
0,640100,Accounting & Financial Statement Analysis: Com...,https://www.udemy.com/accounting-fsa-a-solid-f...,True,150.0,10042,594,43,All Levels,3 hours,2015-10-22T00:03:48Z,BussinessFinance,3.0,2015-10-22,All Levels,True
1,385604,Introduction to Financial Consolidation under ...,https://www.udemy.com/introduction-to-financia...,True,25.0,21,3,8,All Levels,1.5 hours,2016-12-05T14:18:39Z,BussinessFinance,1.5,2016-12-05,All Levels,True
2,834836,How to Consistently Win Trading Stocks in 30 D...,https://www.udemy.com/winningstocktrades/,True,145.0,1433,169,15,Intermediate Level,1 hour,2016-05-09T05:44:33Z,BussinessFinance,1.0,2016-05-09,Intermediate Level,True
3,504620,Mortgage Acceleration,https://www.udemy.com/mortgage-acceleration/,True,20.0,247,2,17,All Levels,1.5 hours,2015-08-21T18:36:25Z,BussinessFinance,1.5,2015-08-21,All Levels,True
4,359926,UK Tax Returns with HMRC,https://www.udemy.com/corporation-tax-returns-...,True,40.0,2,0,11,Beginner Level,1 hour,2016-04-05T15:48:32Z,BussinessFinance,1.0,2016-04-05,Beginner Level,True


In [8]:
df_courseSampling = df_coursesSampling.join(df_scrapedData, how='outer')

In [9]:
df_courseSampling.tail()

,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,contentInfo,...,publishDate,level,paidBool,description,rating,audience,counter,course,requirements,language
1995,1011550,Build Sign Up and Login Forms With Bootstrap M...,https://www.udemy.com/build-sign-up-and-login-...,True,20.0,1898,19,31,All Levels,4 hours,...,2016-11-17,All Levels,True,Add Sign Up ModalAdding Sign Up Form To ModalS...,4.3,['This course is for anyone who wants to learn...,31.0,1011550.0,Basic HTML/CSSMYSQL insert and select queriesT...,English
1996,143028,Code a Responsive Website Using HTML5 and CSS ...,https://www.udemy.com/how-to-code-a-responsive...,True,50.0,1271,136,110,Beginner Level,7.5 hours,...,2014-01-09,Beginner Level,True,Course Overview This course is the equivalen...,4.4,['This course is designed for beginners who wa...,407.0,143028.0,Students will need to download a free copy of ...,English
1997,1179104,Learning Path: React: Make Stunning React Webs...,https://www.udemy.com/learning-path-react-make...,True,200.0,91,5,53,Expert Level,6.5 hours,...,2017-04-18,Expert Level,True,Packt’s Video Learning Paths are a series of i...,3.7,['This course is ideal for web developers. In ...,25.0,1179104.0,Requires working knowledge of ReactJS and some...,English
1998,361620,Wordpress Tutorial,https://www.udemy.com/responsive-design/,True,200.0,2311,8,44,All Levels,1.5 hours,...,2014-12-04,All Levels,True,Are you looking for a step by step video tuto...,4.2,"['Any one who is eager to succeed', 'whoever w...",11.0,361620.0,just a computer a brain and determinationyou d...,English
1999,294408,Learn to Make an Animated Image Gallery using ...,https://www.udemy.com/learn-to-make-an-animate...,False,0.0,11080,165,7,Beginner Level,1 hour,...,2014-11-20,Beginner Level,False,A short and sweet course for all the HTML5 fa...,4.5,"['Front end web develoeprs', 'Web Designers']",326.0,294408.0,"Basic Knowledge of HTML, CSS and JavaScript",English


In [10]:
df_courseSampling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 23 columns):
id                      2000 non-null int64
title                   2000 non-null object
url                     2000 non-null object
isPaid                  2000 non-null bool
price                   2000 non-null float64
numSubscribers          2000 non-null int64
numReviews              2000 non-null int64
numPublishedLectures    2000 non-null int64
instructionalLevel      2000 non-null object
contentInfo             2000 non-null object
publishedTime           2000 non-null object
category                2000 non-null object
timeSpent               2000 non-null float64
publishDate             2000 non-null object
level                   2000 non-null object
paidBool                2000 non-null bool
description             1567 non-null object
rating                  1589 non-null float64
audience                2000 non-null object
counter                 1589 non-null float

In [11]:
df_courseSampling.dropna(inplace=True)

In [12]:
df_courseSampling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1999
Data columns (total 23 columns):
id                      1460 non-null int64
title                   1460 non-null object
url                     1460 non-null object
isPaid                  1460 non-null bool
price                   1460 non-null float64
numSubscribers          1460 non-null int64
numReviews              1460 non-null int64
numPublishedLectures    1460 non-null int64
instructionalLevel      1460 non-null object
contentInfo             1460 non-null object
publishedTime           1460 non-null object
category                1460 non-null object
timeSpent               1460 non-null float64
publishDate             1460 non-null object
level                   1460 non-null object
paidBool                1460 non-null bool
description             1460 non-null object
rating                  1460 non-null float64
audience                1460 non-null object
counter                 1460 non-null float

In [13]:
df_courseSampling.description.head()

0    Accounting is one of the most important skills...
1    This course is an introduction to the financia...
2    *Course Fully Updated for May 2019*The don’t c...
3    This Mortgage Acceleration course will teach y...
4    This course is for bookkeepers, accountants an...
Name: description, dtype: object

In [14]:
df_courseSampling.description[0]

"Accounting is one of the most important skills for people pursuing a career in Finance.It helps you understand whether a business is profitable.It gives you an idea of a company’s size.It helps you use the past in order to take action in the present and change the future.However, it’s essential that you understand it well. If you want to become…a Financial Analystan Accountantan Auditora Business Analysta Financial Controllera Financial Managera CFOa CEOan Investment Bankeran Equity Research Analystan Investor an Entrepreneur Someone who is involved with a business and would like to be successfulThen you simply have to learn Accounting and Financial Statement Analysis. There is no way around it.But how can you do that if you have very limited time and no prior training? And how can you be sure that you are not missing an important piece of the puzzle?Accounting &amp; Financial Statement Analysis: Complete Training is here for you. One of the best Finance courses available on Udemy, it

## Natural Language pre-processing 

### 1. Language detection

In [15]:
def english_detection(string):
    if detect(string) == 'en':
        return True
    else:
        return False

### 2. Building **expandContractions** function

In [16]:
"""
from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
all credits go to alko and arturomp @ stack overflow.
"""

with open('../Data/nlp/wordLists/contractionsList.txt', 'r') as f:
    cList = json.loads(f.read())
    c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [17]:
wpt = nltk.WordPunctTokenizer()
lemmatizer = WordNetLemmatizer() 
stemmer = PorterStemmer() 

### 3. **Corpus normalized** of course descriptions

Let's start to normalize the course description of the row 0, whose name is *Accounting & Financial Statement Analysis: Complete Training*

In [18]:
df_courseSampling.title[0]

'Accounting & Financial Statement Analysis: Complete Training'

**Original description**:

In [19]:
df_courseSampling.description[0]

"Accounting is one of the most important skills for people pursuing a career in Finance.It helps you understand whether a business is profitable.It gives you an idea of a company’s size.It helps you use the past in order to take action in the present and change the future.However, it’s essential that you understand it well. If you want to become…a Financial Analystan Accountantan Auditora Business Analysta Financial Controllera Financial Managera CFOa CEOan Investment Bankeran Equity Research Analystan Investor an Entrepreneur Someone who is involved with a business and would like to be successfulThen you simply have to learn Accounting and Financial Statement Analysis. There is no way around it.But how can you do that if you have very limited time and no prior training? And how can you be sure that you are not missing an important piece of the puzzle?Accounting &amp; Financial Statement Analysis: Complete Training is here for you. One of the best Finance courses available on Udemy, it

Before extract meaningful words, there is a lot of work to do. We note the lack of spaces between miex words; the use of contractions and stopwords. Let's define a function that:
1. Expand contractions
2. Incorporate an extra spaces between mixed words.
3. Remove special characters (once they have been isolated)
4. Apply tokenization
5. Filtering stop-words
6. Apply lemmatization to get the root word

In [20]:
def corpus_normalization(original_text, stop_words, wordlist=False):
    """
    This function receives the origin text and return a corpus normalized. If the stop words are updated, 
    we exclude the word 'no'. In requirements, the difference between 'No previous knowlegde required' 
    and 'previous knowlegde required' is crucial. In description of courses, we are looking for key-words 
    and the edition of the stop words from nltk is not necessary. 
    """
    # Mixed words have uppercase after lowercase letters. Ex:learnedExcellent
    pattern = '[a-z][A-Z]'
    # Return empty string if input is not string, np.nan, None
    if (original_text is None) or (type(original_text) is not str) or (original_text in ['None', 'NIL']):
        return ''
    # Return empty string if input is a non-english string:
    elif english_detection(original_text) is False:
        return ''
    else:
        text = re.sub(r'’',"'", original_text)
        text = expandContractions(text)
        word_list = wpt.tokenize(text)
        # Incorporate extra spaces between mixed words
        words_edit = []
        for word in word_list:
            if re.search(pattern, word): 
                index = re.search(pattern, word).start()
                word = word[:index+1] + ' ' + word[index+1:]
            words_edit.append(word)

        text_filtered = ' '.join(words_edit)
        # Filtering special characters
        text_filtered  = re.sub(r'[^a-zA-Z\s]','', text_filtered)
        text_filtered  = text_filtered.lower()
        # Tokenization to filter stopwords and retrieve roots from derivated words
        tokens = wpt.tokenize(text_filtered)
        words_lem = [lemmatizer.lemmatize(word) for word in tokens if lemmatizer.lemmatize(word) not in stop_words and len(word) > 3]
        if wordlist:
            return words_lem
        text_norm = ' '.join(words_lem)
        
    return text_norm

In [21]:
stop_words = stopwords.words('english')
stop_words.extend(['udemy', 'course', 'school', 'lesson', 'ucstrong', 'rating', 'time', 
                   'html', 'student', 'section', 'professional', 'also', 'using', 'want', 
                   'make', 'take', 'need', 'easy', 'free', 'help', 'basic', 'lecture'])

In [22]:
df_courseSampling['normalized_descriptions'] = df_courseSampling.description.apply(corpus_normalization, stop_words=stop_words)

In [23]:
df_courseSampling['normalized_descriptions'].head()

0    accounting important skill people pursuing car...
1    introduction financial consolidation ifrs aim ...
2    fully updated call trading profit nothing lite...
3    mortgage acceleration teach mortgage work beat...
4    bookkeeper accountant limited company director...
Name: normalized_descriptions, dtype: object

### Corpus normalized of course requirements

In [24]:
to_remove = ['no']
update_stop_words = set(stopwords.words('english')).difference(to_remove)

In [25]:
df_courseSampling['normalized_requirements'] = df_courseSampling.requirements.apply(corpus_normalization, stop_words=update_stop_words)

In [26]:
df_courseSampling['normalized_requirements'].head()

0    prior knowledge accounting assumed needed noth...
1                                 knowledge accounting
2    need basic knowledge stock option trading need...
3    student need reliable computer internet connec...
4    would helpful understood accounting terminolog...
Name: normalized_requirements, dtype: object

In [27]:
df_courseSampling.loc[:, ['description', 'normalized_descriptions', 'requirements', 'normalized_requirements']].head()

,description,normalized_descriptions,requirements,normalized_requirements
0,Accounting is one of the most important skills...,accounting important skill people pursuing car...,No prior knowledge of accounting is assumed or...,prior knowledge accounting assumed needed noth...
1,This course is an introduction to the financia...,introduction financial consolidation ifrs aim ...,some knowledge of accounting,knowledge accounting
2,*Course Fully Updated for May 2019*The don’t c...,fully updated call trading profit nothing lite...,You will need some basic knowledge of stock an...,need basic knowledge stock option trading need...
3,This Mortgage Acceleration course will teach y...,mortgage acceleration teach mortgage work beat...,Students will need a reliable computer and int...,student need reliable computer internet connec...
4,"This course is for bookkeepers, accountants an...",bookkeeper accountant limited company director...,It would be helpful if you understood accounti...,would helpful understood accounting terminolog...


In [28]:
df_courseSampling.reset_index(inplace=True)

## Topic Modeling using Latent Dirichlet Allocation (LDA)

The first purpose is to recognize the topic of every course: 60% of the dataset is used to train a LDA model with the description of courses, chosing courses from the four different categories **Web Development**, **Graphic Design**, **Music and Instrument** and **Bussiness Finance** to avoid unbalanced or biased training dataset.
In this section, we use gensim pachage to generate our Bag of Words.

In [29]:
df_english = df_courseSampling[df_courseSampling.normalized_descriptions != ''].reindex()

In [30]:
df_english.groupby('category').count()

,index,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,...,paidBool,description,rating,audience,counter,course,requirements,language,normalized_descriptions,normalized_requirements
category,,,,,,,,,,,,,,,,,,,,,
BussinessFinance,327,327,327,327,327,327,327,327,327,327,...,327,327,327,327,327,327,327,327,327,327
GraphicDesign,305,305,305,305,305,305,305,305,305,305,...,305,305,305,305,305,305,305,305,305,305
MusicInstrument,368,368,368,368,368,368,368,368,368,368,...,368,368,368,368,368,368,368,368,368,368
WebDevelopment,401,401,401,401,401,401,401,401,401,401,...,401,401,401,401,401,401,401,401,401,401


In [31]:
df_train = df_english.groupby('category').apply(lambda x: x.sample(frac=0.6, random_state=1)).reset_index(drop=True)

In [32]:
df_train.groupby('category').count()

,index,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,...,paidBool,description,rating,audience,counter,course,requirements,language,normalized_descriptions,normalized_requirements
category,,,,,,,,,,,,,,,,,,,,,
BussinessFinance,196,196,196,196,196,196,196,196,196,196,...,196,196,196,196,196,196,196,196,196,196
GraphicDesign,183,183,183,183,183,183,183,183,183,183,...,183,183,183,183,183,183,183,183,183,183
MusicInstrument,221,221,221,221,221,221,221,221,221,221,...,221,221,221,221,221,221,221,221,221,221
WebDevelopment,241,241,241,241,241,241,241,241,241,241,...,241,241,241,241,241,241,241,241,241,241


In [33]:
df_train.head()

,index,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,...,paidBool,description,rating,audience,counter,course,requirements,language,normalized_descriptions,normalized_requirements
0,89,455452,"Accounting, Finance and Banking - A Comprehens...",https://www.udemy.com/accounting-finance-and-b...,True,180.0,507,36,395,All Levels,...,True,"Welcome to this dream course ""Accounting, Fina...",3.9,"['Accounting Students', 'Banking Students', 'F...",168.0,455452.0,This course will teach you from basic to advan...,English,welcome dream accounting finance banking compr...,course teach basic advanced concept hence take...
1,391,302562,Introduction to Accounting: The Language of Bu...,https://www.udemy.com/learnaccountingforfree/,True,20.0,11958,370,134,Beginner Level,...,True,Learn accounting from the self-made millionair...,4.6,"['Students', ' Entrepreneurs', ' and anyone wh...",1470.0,302562.0,"Elementary math skills (i.e., basic addition s...",English,learn accounting self made millionaire norm ne...,elementary math skill basic addition subtracti...
2,419,1273896,Covered Calls - Powerful Income Strategy for S...,https://www.udemy.com/covered-calls-income-str...,True,60.0,22,0,8,Beginner Level,...,True,If you're a Stock trader or Long term stock in...,4.2,"['All Stock Traders', 'Self-directed investors...",33.0,1273896.0,Trading Stocks and holding a stock portfolioAl...,English,stock trader long term stock investor hold sto...,trading stock holding stock portfolio although...
3,341,1045726,Stock Market investment:Non financial fundamen...,https://www.udemy.com/stock-market-investmentn...,True,125.0,1091,4,29,Beginner Level,...,True,"As an investor, how often do you findyourself ...",3.3,['Investors who wish to do their own stock mar...,28.0,1045726.0,Having some knowledge of accounting would be g...,English,investor often findyourself lost read company ...,knowledge accounting would good mandatory
4,190,42643,Master Iron Condors - Double the credit for ha...,https://www.udemy.com/iron-condor-options-trad...,True,60.0,1338,139,7,Expert Level,...,True,THE IRON CONDOR STRATEGY - KING OF TIME DECAY ...,4.4,"['This is an advanced level course', ' so stud...",290.0,42643.0,Excellent knowledge of Credit spreads and all ...,English,iron condor strategy king decay strategiesthe ...,excellent knowledge credit spread material exc...


In [34]:
df_english.groupby('category').count()

,index,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,...,paidBool,description,rating,audience,counter,course,requirements,language,normalized_descriptions,normalized_requirements
category,,,,,,,,,,,,,,,,,,,,,
BussinessFinance,327,327,327,327,327,327,327,327,327,327,...,327,327,327,327,327,327,327,327,327,327
GraphicDesign,305,305,305,305,305,305,305,305,305,305,...,305,305,305,305,305,305,305,305,305,305
MusicInstrument,368,368,368,368,368,368,368,368,368,368,...,368,368,368,368,368,368,368,368,368,368
WebDevelopment,401,401,401,401,401,401,401,401,401,401,...,401,401,401,401,401,401,401,401,401,401


In [35]:
print('Number of documents: {}'.format(len(df_train)))

Number of documents: 841


In [36]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [37]:
# !{sys.executable} -m pip install pyLDAvis
# !{sys.executable} -m pip install -U gensim

In [38]:
# !{sys.executable} -m pip install paramiko

In [39]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, TfidfModel, LdaMulticore

In [40]:
# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [41]:
df_train['wordList_description'] = df_train.description.apply(corpus_normalization, stop_words=stop_words, wordlist=True)

In [42]:
data_words = list(df_train['wordList_description'])

In [43]:
data_words[1]

['learn',
 'accounting',
 'self',
 'made',
 'millionaire',
 'norm',
 'nemrow',
 'recipient',
 'famed',
 'teaching',
 'award',
 'president',
 'united',
 'state',
 'produced',
 'accounting',
 'university',
 'world',
 'brigham',
 'young',
 'university',
 'rated',
 'london',
 'financial',
 'teach',
 'fundamental',
 'financial',
 'accounting',
 'better',
 'effectively',
 'available',
 'today',
 'guaranteed',
 'talk',
 'good',
 'game',
 'gold',
 'standard',
 'specific',
 'prepares',
 'recruit',
 'four',
 'accounting',
 'firm',
 'worldwide',
 'recommended',
 'harvard',
 'incoming',
 'join',
 'university',
 'world',
 'famous',
 'making',
 'available',
 'accounting',
 'knowledge',
 'highlighted',
 'wired',
 'magazine',
 'gagaom',
 'york',
 'popular',
 'well',
 'acclaimed',
 'welcome',
 'norm',
 'nemrow',
 'accounting',
 'nnac',
 'accounting',
 'introduction',
 'accounting',
 'series',
 'composed',
 'five',
 'challenge',
 'learn',
 'content',
 'five',
 'mastered',
 'first',
 'year',
 'accounting

### Bag of words: 

Then, a dictionary from the corpus normalized is created to count the number of times a word appears in the training set:

In [44]:
# Create Dictionary
dictionary = corpora.Dictionary(data_words)

Filtering extremes tokens:
Let's filter tokens that appears in less than 5% of the documents and more than 50% of them.

In [45]:
dictionary.filter_extremes(no_below=40, no_above=0.6)

In [46]:
# Create Corpus
texts = data_words

Using **gensim.doc2bow** we create a dictionary for each document reporting how many words and times the words appear there. This is called corpus and the first corpus is printed as index and then as readable words:

In [47]:
# Term Document Frequency
bow_corpus = [dictionary.doc2bow(text) for text in texts]

print(bow_corpus[:1])

[[(0, 1), (1, 1), (2, 16), (3, 2), (4, 5), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 6), (20, 5), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 4), (32, 2), (33, 1), (34, 1), (35, 1), (36, 3), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 2), (45, 1), (46, 1), (47, 2), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (53, 2), (54, 5), (55, 2), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1)]]


In [48]:
# Human readable format of corpus (term-frequency)
[[(dictionary[id], freq) for id, freq in cp] for cp in bow_corpus[:1]]

[[('access', 1),
  ('account', 1),
  ('accounting', 16),
  ('advanced', 2),
  ('analysis', 5),
  ('answer', 1),
  ('aspect', 1),
  ('beginner', 1),
  ('best', 1),
  ('business', 1),
  ('cannot', 1),
  ('case', 1),
  ('complete', 1),
  ('comprehensive', 1),
  ('concept', 2),
  ('content', 1),
  ('cost', 1),
  ('curriculum', 1),
  ('final', 1),
  ('finance', 6),
  ('financial', 5),
  ('function', 2),
  ('gain', 1),
  ('general', 1),
  ('going', 1),
  ('individual', 1),
  ('introduction', 1),
  ('investment', 1),
  ('knowledge', 1),
  ('learning', 1),
  ('list', 1),
  ('management', 4),
  ('minute', 2),
  ('money', 1),
  ('name', 1),
  ('practical', 1),
  ('process', 3),
  ('purpose', 1),
  ('question', 1),
  ('reading', 1),
  ('right', 1),
  ('self', 1),
  ('single', 1),
  ('skill', 1),
  ('standard', 2),
  ('statement', 1),
  ('strong', 1),
  ('structured', 2),
  ('study', 2),
  ('style', 1),
  ('sure', 1),
  ('system', 1),
  ('teach', 1),
  ('term', 2),
  ('three', 5),
  ('type', 2),
 

### TF-IDF

TF-IDF model from gensim.models:

In [49]:
tfidf = TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [50]:
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus_tfidf[:1]]

[[('access', 0.028623531416995796),
  ('account', 0.046116136121971775),
  ('accounting', 0.7990731291988038),
  ('advanced', 0.05951770150761459),
  ('analysis', 0.22314381944674613),
  ('answer', 0.04249302338781019),
  ('aspect', 0.04917846946123628),
  ('beginner', 0.022831476234553644),
  ('best', 0.02306639379363674),
  ('business', 0.031489414193123316),
  ('cannot', 0.045208005140814835),
  ('case', 0.04809445905444842),
  ('complete', 0.02534306083414552),
  ('comprehensive', 0.04326245554883221),
  ('concept', 0.053120080752253794),
  ('content', 0.024635742358912326),
  ('cost', 0.041052455402932016),
  ('curriculum', 0.04809445905444842),
  ('final', 0.051156792393712916),
  ('finance', 0.2677725833360953),
  ('financial', 0.18139036151310697),
  ('function', 0.08600408417887148),
  ('gain', 0.044346465955620064),
  ('general', 0.050337296613885335),
  ('going', 0.03242650437399576),
  ('individual', 0.04774813066770605),
  ('introduction', 0.03753613951752721),
  ('investm

### Running LDA using Bow and TFIDF corpus

In [88]:
ldaMulticore_model = LdaMulticore(bow_corpus, num_topics=4, id2word=dictionary, passes=20, workers=2, random_state=2)

In [89]:
ldaMulticore_model.print_topics()

[(0,
  '0.023*"website" + 0.016*"create" + 0.014*"development" + 0.013*"code" + 0.013*"project" + 0.013*"build" + 0.012*"application" + 0.011*"data" + 0.010*"step" + 0.010*"type"'),
 (1,
  '0.038*"music" + 0.038*"play" + 0.033*"chord" + 0.027*"guitar" + 0.022*"playing" + 0.020*"piano" + 0.019*"song" + 0.016*"video" + 0.015*"practice" + 0.014*"scale"'),
 (2,
  '0.044*"design" + 0.021*"java" + 0.021*"script" + 0.021*"photoshop" + 0.019*"create" + 0.016*"graphic" + 0.014*"tool" + 0.013*"project" + 0.013*"image" + 0.011*"logo"'),
 (3,
  '0.025*"trading" + 0.018*"financial" + 0.016*"market" + 0.015*"money" + 0.014*"stock" + 0.012*"option" + 0.012*"strategy" + 0.012*"trade" + 0.011*"work" + 0.011*"accounting"')]

Let's calculate the perplexity and coherence score, two values used to evaluate quantitatively topic models. 

In [90]:
# Compute Perplexity
print('\nPerplexity: ', ldaMulticore_model.log_perplexity(bow_corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldaMulticore_model, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.781889194530909

Coherence Score:  0.5364998191327572


In [91]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldaMulticore_model, bow_corpus, dictionary)
vis

/Users/daniela/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.102243  0.023527       1        1  29.408852
3      0.093899  0.115868       2        1  25.101072
1      0.114134 -0.105587       3        1  23.131670
2     -0.105791 -0.033809       4        1  22.358414, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
457   Default  860.000000       design  860.000000  30.0000  30.0000
385   Default  758.000000         play  758.000000  29.0000  29.0000
502   Default  594.000000        chord  594.000000  28.0000  28.0000
508   Default  834.000000        music  834.000000  27.0000  27.0000
514   Default  491.000000       guitar  491.000000  26.0000  26.0000
200   Default  491.000000      trading  491.000000  25.0000  25.0000
506   Default  362.000000    photoshop  362.000000  24.0000  24.0000
478   Default  565.000000      website  565.000000  23.0000  23.0000
517   Default  378.000000         java  378.000000  22.0000  22.0000
500   Default  420.000000      playing  420.000000  21.0000  21.0000
515   Default  356.000000        piano  356.000000  20.0000  20.0000
473   Default  387.000000       script  387.000000  19.0000  19.0000
487   Default  354.000000         song  354.000000  18.0000  18.0000
20    Default  357.000000    financial  357.000000  17.0000  17.0000
437   Default  298.000000      graphic  298.000000  16.0000  16.0000
90    Default  326.000000       market  326.000000  15.0000  15.0000
138   Default  278.000000        stock  278.000000  14.0000  14.0000
323   Default  538.000000      project  538.000000  13.0000  13.0000
410   Default  266.000000        scale  266.000000  12.0000  12.0000
282   Default  763.000000       create  763.000000  11.0000  11.0000
187   Default  352.000000  development  352.000000  10.0000  10.0000
140   Default  230.000000        trade  230.000000   9.0000   9.0000
139   Default  247.000000     strategy  247.000000   8.0000   8.0000
430   Default  349.000000         code  349.000000   7.0000   7.0000
182   Default  260.000000       option  260.000000   6.0000   6.0000
2     Default  215.000000   accounting  215.000000   5.0000   5.0000
159   Default  293.000000        image  293.000000   4.0000   4.0000
483   Default  246.000000         logo  246.000000   3.0000   3.0000
141   Default  182.000000       trader  182.000000   2.0000   2.0000
411   Default  330.000000  application  330.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
497    Topic4   75.010384        color   92.444214   1.2890  -5.4579
129    Topic4   59.236652       medium   78.402374   1.2177  -5.6940
159    Topic4  221.238434        image  293.410889   1.2156  -4.3763
489    Topic4   73.206085        shape   98.778519   1.1984  -5.4823
507    Topic4  102.347672       object  139.386383   1.1891  -5.1472
462    Topic4   62.836796      digital   87.387955   1.1682  -5.6350
174    Topic4   84.819023         text  121.070686   1.1421  -5.3350
112    Topic4  243.412201         tool  382.483521   1.0460  -4.2808
328    Topic4  189.268356        cover  314.840454   0.9891  -4.5324
212    Topic4   62.151112      element  103.535629   0.9876  -5.6460
223    Topic4  125.305794         book  209.996063   0.9816  -4.9448
97     Topic4  100.516243        point  171.829620   0.9618  -5.1652
275    Topic4  120.741066         file  208.094467   0.9536  -4.9819
308    Topic4  154.207474     creating  273.522339   0.9249  -4.7372
389    Topic4   95.570869     software  172.363953   0.9082  -5.2157
208    Topic4   40.344952   background   72.933197   0.9059  -6.0781
99     Topic4   39.247242    principle   71.412216   0.8994  -6.1057
282    Topic4  328.940552       create  763.384155   0.6561  -3.9797
323    Topic4  231.808777      project  538.306152   0.6555  -4.3296
43     Topic4  187.758118        skill  557.0

Save the model to disk importing `datapath`:

In [92]:
from gensim.test.utils import datapath

# temp_file = datapath('gensim_ldaMulticoreModel.csv')
# coherence_model_lda.save(temp_file)

In [93]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=4, id2word=dictionary, passes=20, workers=4, random_state=0)

In [94]:
lda_model_tfidf.print_topics()

[(0,
  '0.025*"design" + 0.021*"photoshop" + 0.015*"graphic" + 0.014*"logo" + 0.013*"illustrator" + 0.012*"image" + 0.011*"designer" + 0.011*"adobe" + 0.010*"tool" + 0.009*"create"'),
 (1,
  '0.018*"website" + 0.013*"java" + 0.013*"script" + 0.013*"code" + 0.012*"application" + 0.011*"development" + 0.009*"create" + 0.008*"project" + 0.008*"page" + 0.008*"developer"'),
 (2,
  '0.029*"guitar" + 0.026*"music" + 0.023*"play" + 0.023*"chord" + 0.021*"piano" + 0.017*"playing" + 0.016*"song" + 0.012*"scale" + 0.011*"blue" + 0.010*"rhythm"'),
 (3,
  '0.023*"trading" + 0.017*"financial" + 0.013*"market" + 0.013*"stock" + 0.012*"trade" + 0.012*"accounting" + 0.012*"strategy" + 0.010*"trader" + 0.010*"tune" + 0.009*"option"')]

In [95]:
# Compute Perplexity
print('\nPerplexity: ', lda_model_tfidf.log_perplexity(corpus_tfidf))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_lda_model_tfidf = CoherenceModel(model=lda_model_tfidf, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda_model_tfidf = coherence_lda_model_tfidf.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_tfidf)


Perplexity:  -6.822714763826055

Coherence Score:  0.7063324018754918


In [96]:
pyLDAvis.enable_notebook()
vis_tfidf = pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, dictionary)
vis_tfidf

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.092997  0.010183       1        1  30.358461
3      0.040780 -0.107946       2        1  24.550127
0     -0.052436  0.031079       3        1  22.663965
2      0.104653  0.066684       4        1  22.427444, topic_info=     Category       Freq         Term      Total  loglift  logprob
term                                                              
514   Default  30.000000       guitar  30.000000  30.0000  30.0000
200   Default  27.000000      trading  27.000000  29.0000  29.0000
502   Default  23.000000        chord  23.000000  28.0000  28.0000
457   Default  32.000000       design  32.000000  27.0000  27.0000
506   Default  22.000000    photoshop  22.000000  26.0000  26.0000
515   Default  22.000000        piano  22.000000  25.0000  25.0000
385   Default  28.000000         play  28.000000  24.0000  24.0000
508   Default  33.000000        music  33.000000  23.0000  23.0000
20    Default  19.000000    financial  19.000000  22.0000  22.0000
478   Default  27.000000      website  27.000000  21.0000  21.0000
487   Default  18.000000         song  18.000000  20.0000  20.0000
500   Default  19.000000      playing  19.000000  19.0000  19.0000
437   Default  17.000000      graphic  17.000000  18.0000  18.0000
483   Default  16.000000         logo  16.000000  17.0000  17.0000
517   Default  19.000000         java  19.000000  16.0000  16.0000
138   Default  15.000000        stock  15.000000  15.0000  15.0000
473   Default  19.000000       script  19.000000  14.0000  14.0000
504   Default  13.000000  illustrator  13.000000  13.0000  13.0000
140   Default  14.000000        trade  14.000000  12.0000  12.0000
430   Default  18.000000         code  18.000000  11.0000  11.0000
90    Default  16.000000       market  16.000000  10.0000  10.0000
410   Default  14.000000        scale  14.000000   9.0000   9.0000
2     Default  14.000000   accounting  14.000000   8.0000   8.0000
139   Default  14.000000     strategy  14.000000   7.0000   7.0000
486   Default  12.000000        adobe  12.000000   6.0000   6.0000
485   Default  11.000000         blue  11.000000   5.0000   5.0000
501   Default  13.000000     designer  13.000000   4.0000   4.0000
141   Default  12.000000       trader  12.000000   3.0000   3.0000
159   Default  15.000000        image  15.000000   2.0000   2.0000
512   Default  10.000000       rhythm  10.000000   1.0000   1.0000
...       ...        ...          ...        ...      ...      ...
487    Topic4  16.718384         song  18.544123   1.3912  -4.1193
503    Topic4   6.412195       finger   7.119450   1.3903  -5.0776
463    Topic4   7.115204       player   7.955734   1.3832  -4.9735
516    Topic4   4.872851       melody   5.558002   1.3633  -5.3521
500    Topic4  17.027237      playing  19.467190   1.3610  -4.1010
412    Topic4   8.111903   instrument   9.372068   1.3505  -4.8424
385    Topic4  24.130335         play  28.910778   1.3141  -3.7523
496    Topic4   6.193642         band   7.647359   1.2840  -5.1122
196    Topic4   7.507136        sound   9.367678   1.2735  -4.9199
508    Topic4  26.547068        music  33.373211   1.2661  -3.6568
513    Topic4   7.134282      musical   9.428789   1.2160  -4.9709
380    Topic4   6.493800        major   9.151634   1.1518  -5.0649
415    Topic4   4.794384        track   7.512994   1.0457  -5.3683
254    Topic4   8.663250         note  13.651030   1.0402  -4.7767
447    Topic4   5.614765        piece   8.850796   1.0398  -5.2104
451    Topic4   3.142586         left   5.186289   0.9939  -5.7907
110    Topic4   6.429163     teaching  10.689981   0.9864  -5.0749
85     Topic4   4.182436         lead   7.118728   0.9630  -5.5049
109    Topic4   5.011502      teacher   8.539683   0.9619  -5.3240
251    Topic4   7.568861     exercise  13.442481   0.9205  -4.9117
219    Topic4   7.657445     practice  14.029490   0.8894  -4.9001
123    Topic4  

In [97]:
# temp_file_tfidf = datapath('gensim_ldaMulticoreTFIDFModel.csv')
# coherence_model_lda.save(temp_file_tfidf)

### Performance evaluation Bag of words

In [98]:
def test_generator(id_course):
    if id_course in list(df_train.id):
        return 'train'
    else:
        return 'test'

In [99]:
df_english['type_data'] = df_english.id.apply(test_generator)
df_test = df_english[df_english.type_data == 'test']

In [100]:
df_test.groupby('category').count()

,index,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,...,description,rating,audience,counter,course,requirements,language,normalized_descriptions,normalized_requirements,type_data
category,,,,,,,,,,,,,,,,,,,,,
BussinessFinance,131,131,131,131,131,131,131,131,131,131,...,131,131,131,131,131,131,131,131,131,131
GraphicDesign,122,122,122,122,122,122,122,122,122,122,...,122,122,122,122,122,122,122,122,122,122
MusicInstrument,147,147,147,147,147,147,147,147,147,147,...,147,147,147,147,147,147,147,147,147,147
WebDevelopment,160,160,160,160,160,160,160,160,160,160,...,160,160,160,160,160,160,160,160,160,160


In [101]:
df_test['wordList_description'] = df_test.description.apply(corpus_normalization, stop_words=stop_words, wordlist=True)

/Users/daniela/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [102]:
test_words = list(df_test['wordList_description'])

In [103]:
bow_tests = [dictionary.doc2bow(text) for text in test_words]

In [104]:
bow_tests[0]

[(1, 1),
 (11, 1),
 (20, 5),
 (24, 1),
 (26, 1),
 (36, 1),
 (48, 1),
 (67, 1),
 (161, 1),
 (163, 1),
 (164, 1),
 (217, 1),
 (229, 1),
 (253, 1),
 (293, 1),
 (313, 1),
 (351, 1)]

Let's analyse an first course on the test dataset:

In [105]:
df_test.reset_index(inplace=True)

In [106]:
print('Title: {}'.format(df_test.title[0]))
print('Category: {}'.format(df_test.category[0]))

Title: Introduction to Financial Consolidation under IFRS
Category: BussinessFinance


Likely to belongs to each category:

In [107]:
for index, score in sorted(ldaMulticore_model[bow_tests[0]], key=lambda tup: -1*tup[1]):
    print('Score: {}\t Topic: {}'.format(score, ldaMulticore_model.print_topic(index, 5)))

Score: 0.9638637900352478	 Topic: 0.025*"trading" + 0.018*"financial" + 0.016*"market" + 0.015*"money" + 0.014*"stock"
Score: 0.012210945598781109	 Topic: 0.038*"music" + 0.038*"play" + 0.033*"chord" + 0.027*"guitar" + 0.022*"playing"
Score: 0.011991395615041256	 Topic: 0.044*"design" + 0.021*"java" + 0.021*"script" + 0.021*"photoshop" + 0.019*"create"
Score: 0.011933831498026848	 Topic: 0.023*"website" + 0.016*"create" + 0.014*"development" + 0.013*"code" + 0.013*"project"


In [108]:
print('Title: {}'.format(df_test.title[150]))
print('Category: {}'.format(df_test.category[150]))

Title: Graphic Design Secrets Revealed
Category: GraphicDesign


In [109]:
for index, score in sorted(ldaMulticore_model[bow_tests[150]], key=lambda tup: -1*tup[1]):
    print('Score: {}\t Topic: {}'.format(score, ldaMulticore_model.print_topic(index, 5)))

Score: 0.6190391778945923	 Topic: 0.044*"design" + 0.021*"java" + 0.021*"script" + 0.021*"photoshop" + 0.019*"create"
Score: 0.3788594901561737	 Topic: 0.023*"website" + 0.016*"create" + 0.014*"development" + 0.013*"code" + 0.013*"project"


In [110]:
print('Title: {}'.format(df_test.title[300]))
print('Category: {}'.format(df_test.category[300]))

Title: Learn BASS GUITAR- Have Fun, Amaze Your Friends- It's Easy! 
Category: MusicInstrument


In [111]:
for index, score in sorted(ldaMulticore_model[bow_tests[300]], key=lambda tup: -1*tup[1]):
    print('Score: {}\t Topic: {}'.format(score, ldaMulticore_model.print_topic(index, 5)))

Score: 0.9919224381446838	 Topic: 0.038*"music" + 0.038*"play" + 0.033*"chord" + 0.027*"guitar" + 0.022*"playing"


In [112]:
df_predictions = df_test.copy()

#### Extraction of the dominant topic:

In [113]:
topic_num_list = []
topic_prob_list = []
topic_key_words = []

for i in range(len(df_test)):
    for index, score in enumerate(ldaMulticore_model[bow_tests[i]]):
        row = sorted(ldaMulticore_model[bow_tests[i]], key=lambda tup: -1*tup[1])
        for j, (topic_num, topic_prob) in enumerate(row):
            if j==0:
                wp = ldaMulticore_model.show_topic(topic_num)
                topic_keywords = ', '.join([word for word, prop in wp])
                dominant_topic = topic_num
                dominant_probability = topic_prob
                dominant_keywords = topic_keywords
    topic_num_list.append(int(dominant_topic))
    topic_prob_list.append(round(dominant_probability, 4))
    topic_key_words.append(dominant_keywords)

In [114]:
df_predictions['dominant_topic'] = topic_num_list
df_predictions['topic_prob_list'] = topic_prob_list
df_predictions['topic_key_words'] = topic_key_words

In [115]:
df_TopicModeling = df_predictions.loc[:, ['title', 'category', 'wordList_description', 'dominant_topic', 'topic_prob_list', 'topic_key_words']]

In [116]:
df_TopicModeling.sample(n=20, random_state=1)

,title,category,wordList_description,dominant_topic,topic_prob_list,topic_key_words
361,Bluegrass Guitar Essentials: Webisodes 1 & 2,MusicInstrument,"[note, made, webisodes, absolutelyfree, enjoy,...",1,0.8951,"music, play, chord, guitar, playing, piano, so..."
517,Bootstrap 3 Beginners: Create a Professional L...,WebDevelopment,"[learn, concept, tool, function, build, fully,...",0,0.9058,"website, create, development, code, project, b..."
349,Flute Lessons For Beginners,MusicInstrument,"[perfect, place, start, beginner, flute, playe...",1,0.9928,"music, play, chord, guitar, playing, piano, so..."
486,Wordpress Security,WebDevelopment,"[staying, ahead, change, wordpress, security, ...",0,0.6749,"website, create, development, code, project, b..."
475,Flexbox: The complete guide,WebDevelopment,"[flexbox, layout, module, let, build, complex,...",0,0.7879,"website, create, development, code, project, b..."
483,"Build Modern Responsive Website With HTML5, CS...",WebDevelopment,"[already, taking, best, reviewed, building, mo...",0,0.9707,"website, create, development, code, project, b..."
29,How I Learned Stock Trading (and How You Can Too),BussinessFinance,"[stock, trading, principle, quote, stock, trad...",3,0.9906,"trading, financial, market, money, stock, opti..."
236,Adobe Illustrator For Everyone: Create Detaile...,GraphicDesign,"[simple, step, stepcourse, graphic, design, il...",2,0.9847,"design, java, script, photoshop, create, graph..."
168,How to create flat design icons in 15 minutes ...,GraphicDesign,"[learn, illustrator, flat, design, rather, str...",2,0.8685,"design, java, script, photoshop, create, graph..."
65,Essential Finance For Managers,BussinessFinance,"[manager, would, manager, baffled, balance, sh...",3,0.8815,"trading, financial, market, money, stock, opti..."


In [117]:
def category2topic(category):
    """ The correlation between categories and dominant topic could change if you get a different topic modeling.
    """
    if category == 'MusicInstrument':
        topic = 1
    elif category == 'GraphicDesign':
        topic = 2
    elif category == 'WebDevelopment':
        topic = 0
    else:
        topic = 3
        
    return topic

In [118]:
df_TopicModeling['true_topic'] = df_TopicModeling.category.apply(category2topic)

In [120]:
df_TopicModeling.sample(n=20, random_state=0)

,title,category,wordList_description,dominant_topic,topic_prob_list,topic_key_words,true_topic
504,"Building a Portfolio in 90 minutes ( HTML, CSS...",WebDevelopment,"[message, coupon, code, create, online, portfo...",0,0.8617,"website, create, development, code, project, b...",0
408,"Web Application Development using Redis, Expre...",WebDevelopment,"[looking, build, scalable, real, application, ...",0,0.7515,"website, create, development, code, project, b...",0
66,Investing 101: The Complete Online Investing C...,BussinessFinance,"[attention, investor, wealth, seeker, stock, m...",3,0.8617,"trading, financial, market, money, stock, opti...",3
338,Learn To Play The Drums,MusicInstrument,"[hello, welcome, learn, play, drum, seller, bi...",1,0.6412,"music, play, chord, guitar, playing, piano, so...",1
233,The Ultimate Drawing Course - Beginner to Adva...,GraphicDesign,"[join, learning, start, gaining, drawing, skil...",0,0.4414,"website, create, development, code, project, b...",2
440,WordPress For Beginners | No Coding Required,WebDevelopment,"[code, newyear, special, discount, offer, year...",0,0.6100,"website, create, development, code, project, b...",0
498,A-Frame WebVR Programming Tutorial Series (Vir...,WebDevelopment,"[virtual, reality, dream, science, fiction, in...",0,0.6691,"website, create, development, code, project, b...",0
538,Meteor and React for Realtime Apps,WebDevelopment,"[upvotes, downvotes, implementing, image, scor...",0,0.9850,"website, create, development, code, project, b...",0
355,Piano Lesson Variety,MusicInstrument,"[great, place, start, intermediate, piano, lea...",1,0.9764,"music, play, chord, guitar, playing, piano, so...",1
507,Responsive Web Design with HTML5 and CSS3 - In...,WebDevelopment,"[underpinnings, responsive, page, code, snippe...",0,0.5708,"website, create, development, code, project, b...",0


In [121]:
from sklearn.metrics import confusion_matrix, classification_report

y_pred = df_TopicModeling['dominant_topic']
y_true = df_TopicModeling['true_topic']

Finally, to evaluate the performance of this classification, the confusion matrix is displayed above. Additionally, the most revelant classification metrix (precision, recall, F1 score and support) of every class are showed using the `classification_report` tool from sklearn metrics. 
As a reminder, **precision** is the ratio of correctly predicted positive observations to the total of predicted positive observations. It means, $\frac{TP}{TP+FP}$. **Recall** represents the correctly predicted positive observations to the all positive observations, or $\frac{TP}{TP+FN}$. 

F1 score is useful when false positives and false negatives have similar cost. This metric is a weighted average of precision and recall. The mathematical expression is $2*\frac{Recall*Precision}{Recall + Precition}$. Finally, support is the number of ocurrence of every class.

In [122]:
confusion_matrix(y_true, y_pred)

array([[143,   0,  17,   0],
       [  0, 136,   0,  11],
       [  7,   3, 107,   5],
       [  7,   0,   1, 123]])

In [123]:
target_names = ['class 0', 'class 1', 'class 2', 'class 3']
print(classification_report(y_true, y_pred, labels=[0, 1, 2, 3], target_names=['WebDevelopment', 'MusicInstrument', 
                                                                               'GraphicDesign', 'BussinessFinance']))

                  precision    recall  f1-score   support

  WebDevelopment       0.91      0.89      0.90       160
 MusicInstrument       0.98      0.93      0.95       147
   GraphicDesign       0.86      0.88      0.87       122
BussinessFinance       0.88      0.94      0.91       131

       micro avg       0.91      0.91      0.91       560
       macro avg       0.91      0.91      0.91       560
    weighted avg       0.91      0.91      0.91       560



- Music Instrument courses have the highest precision ratio (98%) and recall ratio (93%). According to the confusion matrix, there are 147 courses in the category of Music and 136 of them are identified correctly (in other words, $FN=11$ and Recall is 93%). 3 Graphic courses are mislabeled as Music (therefore, $FP=3$ and Precision reaches 98%). 

- Graphic Design is the category with more false positives ($FP=18$ and in consequence, is the class with lowest precision rate) and at the same time, is the lowest recall rate with $FN=15$. 

- Web development presents a high precision but at the same time is the second lowest recall rate. It means that, between all the true web courses, the correct classification is around 91%, but the model mislabels courses from other categories as web courses (and the recall rate is 89%). This is the classic trade-off in the classification algorithms: as good is the model recognizing a class, as higher is the risk of including observations from other classes, rising the number of false positives. We sacrifice precision or recall depending on the cost that every metric has in the problem we are trying to resolve. 


#### How we could explain those mistakes? 

- Checking the representative words of every category, `script` and `java` are found as relevant terms on the design topic on the modeling process.
- Design and Web are very close on the Intertopic Distance Map. In the Design sphere we can find terms as `software`, `technology`, `tool`, `project`, `user`, and `technique`. Same expressions are used on Development terminology. Instead, Music and Finance have, in general, more unique and specific terminology that allow us to separate them appropriately (`guitar`, `playing`, `song`, `piano`, `music` versus `market`, `finantial`, `trading`, `strategy`, `stock` just to mention a couple of examples).